In [2]:
from google.colab import drive
import os

drive.mount ('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [2]:
%cd drive/My\ Drive/Colab\ Notebooks


/content/drive/My Drive/Colab Notebooks


In [3]:
%cd project2

/content/drive/My Drive/Colab Notebooks/project2


In [4]:
%ls training/groundtruth

satImage_001.png  satImage_026.png  satImage_051.png  satImage_076.png
satImage_002.png  satImage_027.png  satImage_052.png  satImage_077.png
satImage_003.png  satImage_028.png  satImage_053.png  satImage_078.png
satImage_004.png  satImage_029.png  satImage_054.png  satImage_079.png
satImage_005.png  satImage_030.png  satImage_055.png  satImage_080.png
satImage_006.png  satImage_031.png  satImage_056.png  satImage_081.png
satImage_007.png  satImage_032.png  satImage_057.png  satImage_082.png
satImage_008.png  satImage_033.png  satImage_058.png  satImage_083.png
satImage_009.png  satImage_034.png  satImage_059.png  satImage_084.png
satImage_010.png  satImage_035.png  satImage_060.png  satImage_085.png
satImage_011.png  satImage_036.png  satImage_061.png  satImage_086.png
satImage_012.png  satImage_037.png  satImage_062.png  satImage_087.png
satImage_013.png  satImage_038.png  satImage_063.png  satImage_088.png
satImage_014.png  satImage_039.png  satImage_064.png  satImage_089.png
satIma

In [1]:
from preprocessing import *
from proj2_helpers import *
from UNet_model import *

In [2]:
X_train, X_test, Y_train, Y_test = preprocess(divide_set=True, ratio=0.25)



processing image  0 / 100
processing image  1 / 100
processing image  2 / 100
processing image  3 / 100
processing image  4 / 100
processing image  5 / 100
processing image  6 / 100
processing image  7 / 100
processing image  8 / 100
processing image  9 / 100
processing image  10 / 100
processing image  11 / 100
processing image  12 / 100
processing image  13 / 100
processing image  14 / 100
processing image  15 / 100
processing image  16 / 100
processing image  17 / 100
processing image  18 / 100
processing image  19 / 100
processing image  20 / 100
processing image  21 / 100
processing image  22 / 100
processing image  23 / 100
processing image  24 / 100
processing image  25 / 100
processing image  26 / 100
processing image  27 / 100
processing image  28 / 100
processing image  29 / 100
processing image  30 / 100
processing image  31 / 100
processing image  32 / 100
processing image  33 / 100
processing image  34 / 100
processing image  35 / 100
processing image  36 / 100
processing 

In [3]:
print(Y_train[90].shape)
print(X_train[10].shape)

(400, 400, 1)
(400, 400, 3)


In [4]:
from run import run

result = run(X_train, Y_train, X_test, Y_test)

print(result)

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: lambda/truediv:0


ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_1:0", shape=(None, 400, 400, 3), dtype=float32) at layer "lambda". The following previous layers were accessed without issue: []

In [21]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(img_aug_arr, patch_size, patch_size) for img_aug_arr in imgs_aug_arr]
gt_patches = [img_crop(gt_img_aug_arr, patch_size, patch_size) for gt_img_aug_arr in gt_imgs_aug_arr]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [ ]:
img_patches.shape

(58491,)

In [ ]:
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [ ]:
print('Computed ' + str(X.shape[0]) + ' features')
print('Feature dimension = ' + str(X.shape[1]))
print('Number of classes = ' + str(np.max(Y)))  #TODO: fix, length(unique(Y)) 

Y0 = [i for i, j in enumerate(Y) if j == 0]
Y1 = [i for i, j in enumerate(Y) if j == 1]
print('Class 0: ' + str(len(Y0)) + ' samples')
print('Class 1: ' + str(len(Y1)) + ' samples')

from sklearn import linear_model

# we create an instance of the classifier and fit the data
logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(X, Y)

# Predict on the training set
Z = logreg.predict(X)

# Get non-zeros in prediction and grountruth arrays
Zn = np.nonzero(Z)[0]
Yn = np.nonzero(Y)[0]

TPR = len(list(set(Yn) & set(Zn))) / float(len(Z))
print('True positive rate = ' + str(TPR))

True positive rate = 0.21977740165153614
